In [1]:
# !pip install selenium
# !pip install webdriver_manager
# !pip install sqlalchemy PyMySQL

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
# from tqdm import tqdm
import time
import random
import lxml
import pandas as pd

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
# 출력 행 제한을 없애는 코드
pd.set_option('display.max_rows', 200)

### 날짜 생성 코드

In [5]:
import datetime

start_date = datetime.date(2024, 8, 27)
end_date = datetime.date(2024, 9, 4)

# 월-금요일에 해당하는 날짜 리스트 생성
weekdays = [0, 1, 2, 3, 4]  # 월요일=0, 화요일=1, ..., 금요일=4
date_list = []

current_date = start_date
while current_date <= end_date:
    if current_date.weekday() in weekdays:
        date_list.append(current_date)
    current_date += datetime.timedelta(days=1)

print(date_list)


[datetime.date(2024, 8, 27), datetime.date(2024, 8, 28), datetime.date(2024, 8, 29), datetime.date(2024, 8, 30), datetime.date(2024, 9, 2), datetime.date(2024, 9, 3), datetime.date(2024, 9, 4)]


In [6]:
str(date_list[0])

'2024-08-27'

In [7]:
# 크롬 옵션즈에 User-Agent, lang 같은 정보를 담아 셀레니움을 이용한 크롤링이 아닌 것 처럼 만들기
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
# options.add_argument('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36')
options.add_argument("lang=ko_KR")

# 웹드라이버를 버전에 맞게 자동으로 다운 받고 옵션을 추가해 줌
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.set_window_size(1920,1080)   # 웹브라우저 해상도 조절

driver.get("https://www.hanabank.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do#//HanaBank")  # 크롤링 할 웹사이트 주소

In [8]:
cols13 = ('통화', '현찰_살때_환율', '현찰_살때_Spread', '현찰_팔때_환율', '현찰_팔때_Spread', 
       '송금_보낼때', '송금_받을때', 'T/C_살때', '외화_수표_팔때', '매매기준율', '환가_료율',
       '미화 환산율', 'date')
sorted_cols13 = ('date', '통화', '현찰_살때_환율', '현찰_살때_Spread', '현찰_팔때_환율', '현찰_팔때_Spread', 
       '송금_보낼때', '송금_받을때', 'T/C_살때', '외화_수표_팔때', '매매기준율', '환가_료율',
       '미화 환산율')

In [9]:
cols12 = ('통화', '현찰_살때_환율', '현찰_살때_Spread', '현찰_팔때_환율', '현찰_팔때_Spread', 
       '송금_보낼때', '송금_받을때', '외화_수표_팔때', '매매기준율', '환가_료율', '미화 환산율', 'date')
sorted_cols12 = ('date', '통화', '현찰_살때_환율', '현찰_살때_Spread', '현찰_팔때_환율', '현찰_팔때_Spread', 
       '송금_보낼때', '송금_받을때', '외화_수표_팔때', '매매기준율', '환가_료율', '미화 환산율')

In [10]:
cols11 = ('통화', '현찰_살때_환율', '현찰_살때_Spread', '현찰_팔때_환율', '현찰_팔때_Spread', 
       '송금_보낼때', '송금_받을때', '매매기준율', '환가_료율', '미화 환산율', 'date')
sorted_cols11 = ('date', '통화', '현찰_살때_환율', '현찰_살때_Spread', '현찰_팔때_환율', '현찰_팔때_Spread', 
       '송금_보낼때', '송금_받을때', '매매기준율', '환가_료율', '미화 환산율')

In [11]:
date_list

[datetime.date(2024, 8, 27),
 datetime.date(2024, 8, 28),
 datetime.date(2024, 8, 29),
 datetime.date(2024, 8, 30),
 datetime.date(2024, 9, 2),
 datetime.date(2024, 9, 3),
 datetime.date(2024, 9, 4)]

In [12]:
# for idx, date in enumerate(date_list[:]):
#     # 웹 요소가 나타날 때까지 최대 10초 동안 기다립니다.
#     wait = WebDriverWait(driver, 10)

#     # 날짜 박스 #tmpInqStrDt
#     datebox = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#tmpInqStrDt')))
#     datebox.clear()   # 기존에 입력된 값을 지웁니다.
#     datebox.send_keys(f"{str(date)}")   # 새로운 값을 입력합니다.

#     # 조회버튼   #HANA_CONTENTS_DIV > div.btnBoxCenter > a
#     search_box = driver.find_element(By.CSS_SELECTOR, '#HANA_CONTENTS_DIV > div.btnBoxCenter > a')
#     search_box.click()   # 조회버튼 클릭
#     # 생성한 시간 동안 대기합니다.
#     time.sleep(random.uniform(3, 7))
#     exchange_rate_table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#searchContentDiv > div.printdiv > table')))

#     # 페이지의 전체 HTML 소스를 가져옵니다.
#     page_html = driver.page_source
#     # soup = bs(page_html, "lxml")     # beautifulsoup으로 html을 parsing
#     exchange_rate = pd.read_html(page_html)
#     exchange_rate = exchange_rate[1]
#     exchange_rate['date'] = f"{date}"
    
#     if len(exchange_rate.columns) == 13:
#         exchange_rate.columns = cols13
#         exchange_rate = exchange_rate[[*sorted_cols13]]
#     elif len(exchange_rate.columns) == 12:    
#         exchange_rate.columns = cols12
#         exchange_rate = exchange_rate[[*sorted_cols12]]
#     elif len(exchange_rate.columns) == 11:    
#         exchange_rate.columns = cols11
#         exchange_rate = exchange_rate[[*sorted_cols11]]
    
#     display(exchange_rate)
#     print(f"전체 {len(date_list)} 중 {idx+1:04d} 페이지 완료", end="\r")

    

In [13]:
# exchange_rate.info()

In [14]:
# exchange_rate

# DB에 직접 저장

In [15]:
import dbio

In [18]:
for idx, date in enumerate(date_list[:]):
    # 웹 요소가 나타날 때까지 최대 10초 동안 기다립니다.
    wait = WebDriverWait(driver, 10)

    # 날짜 박스 #tmpInqStrDt
    datebox = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#tmpInqStrDt')))
    datebox.clear()   # 기존에 입력된 값을 지웁니다.
    datebox.send_keys(f"{date}")   # 새로운 값을 입력합니다.

    # 조회버튼   #HANA_CONTENTS_DIV > div.btnBoxCenter > a
    search_box = driver.find_element(By.CSS_SELECTOR, '#HANA_CONTENTS_DIV > div.btnBoxCenter > a')
    search_box.click()   # 조회버튼 클릭
    # 생성한 시간 동안 대기합니다.
    time.sleep(random.uniform(3, 7))
    exchange_rate_table = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#searchContentDiv > div.printdiv > table')))

    # 페이지의 전체 HTML 소스를 가져옵니다.
    page_html = driver.page_source
    # soup = bs(page_html, "lxml")     # beautifulsoup으로 html을 parsing
    exchange_rate_df = pd.read_html(page_html)
    exchange_rate_df = exchange_rate_df[1]
    exchange_rate_df['date'] = f"{date}"
    
    if len(exchange_rate_df.columns) == 13:
        exchange_rate_df.columns = cols13
        exchange_rate_df = exchange_rate_df[[*sorted_cols13]]
    elif len(exchange_rate_df.columns) == 12:    
        exchange_rate_df.columns = cols12
        exchange_rate_df = exchange_rate_df[[*sorted_cols12]]
    elif len(exchange_rate_df.columns) == 11:    
        exchange_rate_df.columns = cols11
        exchange_rate_df = exchange_rate_df[[*sorted_cols11]]
    
    dbio.exi_to_db("exchange_rate", date, exchange_rate_df)
    print(f"전체 {len(date_list)} 중 {idx+1:04d} 페이지 완료", end="\r")

    

In [ ]:
# final_df['date'][0]

In [ ]:
# to_db("exchange_rate", final_df

In [ ]:
# 57*365*10

In [ ]:
# for mon in range(1,13):
#     print(mon)